In [1]:
# from dfender import distribute
# import pandas as pd
# import dask.dataframe as dd

# X = pd.DataFrame(range(100), columns=['x'])
# X_ = dd.from_pandas(X, npartitions=1)

# class Test:
#     @distribute
#     def predict(self, X: pd.Series) -> pd.Series:
#         return (pd.to_datetime('2000-01-01').to_period('M') + X).dt.to_timestamp()

#     @distribute
#     def rowmax(self, X: pd.DataFrame):
#         return X.max(axis=1)
#     # @dispatch(pd.DataFrame)
#     # def predict(self, X: pd.DataFrame) -> pd.Series:
#     #     return X * 2

#     # @dispatch(dd.DataFrame)
#     # def predict(self, X: dd.DataFrame) -> dd.Series:
#     #     return X / 2


# est = Test()
# # est.predict(X['x'])
# # est.predict(X_['x'])
# est.rowmax(X)
# est.rowmax(X_)

In [5]:
from dfender.materializers import materializer

materializer.registry

dask.dataframe.core.Series